<a href="https://colab.research.google.com/github/dhananjayamandal/Signal-Processing/blob/master/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Program based on Digital SIgnal Processing

In [0]:
import numpy as np
from matplotlib import pyplot as plt 
gauss = lambda t:np.exp(-0.2*t*t)

In [0]:
def lowpass(fc):
  return lambda t: 2*fc *np.sinc(2*fc*t)

In [0]:
import numpy as np
from matplotlib import pyplot as plt
x={1,1,1,1}
y={1,2,1,1}
def showConvolution(x,y):
  Fs=100
  T=100
  t=np.arange(-T,T,1/Fs)
  t_convolved=np.arange(-2*T,2*T,1/Fs)[:-1]
  x_samples=x(t)*np.hanning(len(t))
  y_samples=y(t)*np.hanning(len(t))
  z_samples=np.convolve(x_samples,y_samples) / Fs
  
  spec = lambda x: np.fft.fftshift(np.fft(x,16*len(t))) / Fs
  X_samples = spec(x_samples)
  Y_samples = spec(y_samples)
  Z_samples = spec(z_samples)
  f=np.linspace(-Fs/2,Fs/2, len(X_samples),endpoint=False)
  
  plt.subplot(121)
  plt.plot(t,x_samples,label+'$x(t)$')
  plt.plot(t,y_samples,label+'$y(t)$')
  plt.plot(t_convolved,z_samples,label='$z(t)$')
  
  plt.subplot(122)
  plt.plot(f,abs(X_samples),label='$|X(f)|$')
  plt.plot(f,abs(Y_samples),label='$|Y(f)|$')
  plt.plot(f,abs(Z_samples),label='$|Z(f)|$')

In [0]:
import numpy as np
from matplotlib import pyplot as plt

def rect(t):
    return (abs(t) <= 0.5).astype(float)
def gauss(t):
    return np.exp(-t*t)
def triang(t):
    return (1-abs(t)) * rect(t/2)

def ft(samples, Fs, t0):
    """Approximate the Fourier Transform of a time-limited signal 
    by means of the discrete Fourier Transform.
    
    samples: signal values sampled at the positions t0 + n/Fs
    Fs: Sampling frequency of the signal
    t0: starting time of the sampling of the signal
    """
    f = np.linspace(-Fs/2, Fs/2, len(samples), endpoint=False)
    return np.fft.fftshift(np.fft.fft(samples)/Fs * np.exp(-2j*np.pi*f*t0))
  
tstart = 10
Fs = 1000
t = np.arange(-tstart, tstart, 1/Fs)
f = np.arange(-Fs/2, Fs/2, Fs/len(t))

x = rect
y = gauss
X = ft(x(t), Fs, -tstart)
Y = ft(y(t), Fs, -tstart)

def showLinearity(a):
    plt.subplot(221)
    plt.plot(t, x(t), label='$x(t)$')
    plt.plot(t, y(t), label='$y(t)$')

    plt.subplot(222)
    plt.plot(f, X.real, label='$X(f)$')
    plt.plot(f, Y.real, label='$Y(f)$')
    
    xplusay = lambda t: x(t) + a*y(t)
    XplusaY = ft(xplusay(t), Fs, -t0)
    plt.subplot(223)
    plt.plot(t, xplusay(t), '-*', label='$x(t)+ay(t)$', markevery=400)
    plt.plot(t, x(t), label='$x(t)$')
    plt.plot(t, a*y(t), label='$ay(t)$')
    
    plt.subplot(224)
    plt.plot(f, XplusaY.real, '-*', label='$X(f)+aY(f)$', markevery=10)
    plt.plot(f, X.real, label='$X(f)$')
    plt.plot(f, a*Y.real, label='$aY(f)$')

In [2]:
import numpy as np
def DFT_slow(x):
    """Compute the discrete Fourier Transform of the 1D array x"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)
  
x = np.random.random(1024)
np.allclose(DFT_slow(x), np.fft.fft(x))

%timeit DFT_slow(x)
%timeit np.fft.fft(x)

10 loops, best of 3: 108 ms per loop
The slowest run took 18.37 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 27.1 µs per loop


In [3]:
import numpy as np
def FFT(x):
    """A recursive implementation of the 1D Cooley-Tukey FFT"""
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 32:  # this cutoff should be optimized
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate([X_even + factor[:N / 2] * X_odd,
                               X_even + factor[N / 2:] * X_odd])
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

%timeit DFT_slow(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

TypeError: ignored